In [ ]:
env = RushHourEnv(vehicle_data)

# Independent DQN 에이전트 생성
state_size = env.grid_size[0]
action_size = env.action_space.nvec[1]
agents = [DQNAgent(state_size, action_size) for _ in range(len(vehicle_data))]

# Independent DQN 에이전트 생성
state_size = env.grid_size[0]
action_size = env.action_space.nvec[1]
agents = [DQNAgent(state_size, action_size) for _ in range(len(vehicle_data))]

episode_steps = []
episode_rewards = []
successful_episodes = 0
num_episodes = 1000
for episode in range(num_episodes):
    state = env.reset()
    done = False
    total_reward = 0
    steps = 0

    while not done:
        actions = []
        for i, agent in enumerate(agents):
            action = agent.select_action(state)
            actions.append((i, action))

        # 환경에서 행동 수행
        next_state, reward, done, _ = env.step(actions)
        total_reward += reward
        steps += 1

        # 각 에이전트 경험 저장
        for i, agent in enumerate(agents):
            agent.store_experience(state, actions[i][1], reward, next_state, done)

        # 각 에이전트 학습
        for agent in agents:
            agent.train()

        state = next_state

    # Epsilon 감소
    for agent in agents:
        agent.decay_epsilon()

    # Track steps and rewards
    episode_steps.append(steps)
    episode_rewards.append(total_reward)

    # Count if the target vehicle reached the exit
    target_front_position = env.target_vehicle.get_positions()[-1]
    if target_front_position == (env.exit_wall[0], env.exit_wall[1] - 1):
        successful_episodes += 1

    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}")